# Jupyter Notebook - Procesamiento del habla
Primero, vamos a importar los paquetes necesarios para el desarrollo.

In [1]:
%matplotlib inline
import os
import plotly
import wav

import tensorflow as tf
import numpy as np
#import xgboost

from matplotlib import pyplot
from IPython.display import Audio

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Convolution1D, MaxPooling1D, Dropout, Flatten

from plotly.graph_objs import Scatter, Layout, Bar, Figure

plotly.offline.init_notebook_mode()

Using TensorFlow backend.


Comencemos primero definiendo algunas variables globales (parámetros del enunciado).

In [2]:
RUTA_DIRECTORIO_DATOS = "datos"
VENTANA_EN_SEGUNDOS = 10
SAMPLE_RATE = 16000
CANTIDAD_DE_FRAMES_A_PROCESAR = 318764 # Es el minimo de todos en este caso

Ahora vamos a cargar los archivos IPU y WAV

In [3]:
archivos_en_carpeta_datos = os.listdir(RUTA_DIRECTORIO_DATOS)
archivos_wav = []
archivos_ipu = []
for archivo in archivos_en_carpeta_datos:
    if (archivo[-3:] == "ipu"):
        archivos_ipu.append(archivo)
    elif (archivo[-3:] == "wav"):
        archivos_wav.append(archivo)

Creo mi set de datos (X's)

In [4]:
X = []
leng = []
for archivo_wav in archivos_wav:
    data, frames, _, duration = wav.load_from_wav(RUTA_DIRECTORIO_DATOS + "/" + archivo_wav)
    leng.append(len(data))
    X.append(data[:CANTIDAD_DE_FRAMES_A_PROCESAR])

X = np.asarray(X)
print(X.shape)

(184, 318764)


Creo las etiquetas (mis Y's)

In [5]:
y = []
y_sgd = []
for archivo_wav in archivos_wav:
    if archivo_wav[3] == "m":
        y.append([1,0])
        y_sgd.append(1)
    elif archivo_wav[3] == "f":
        y.append([0,1])
        y_sgd.append(0)
y = np.asarray(y)
y_sgd = np.asarray(y_sgd)

print(y.shape)
print(y_sgd.shape)

(184, 2)
(184,)


Testeo que los datos hayan sido cargados correctamente

In [6]:
Audio(data=X[21], rate=SAMPLE_RATE)

In [7]:
print(y[21])

[0 1]


Ahora separo en un set de entrenamiento y testing:

In [8]:
X, y = shuffle(X, y_sgd, random_state=42)
X_train = X[:150]
X_test = X[150:180]
y_train = y[:150]
y_test = y[150:180]

print("Shape X_train {}".format(np.shape(X_train)))
print("Shape X_test {}".format(np.shape(X_test)))
print("Shape y_train {}".format(np.shape(y_train)))
print("Shape y_test {}".format(np.shape(y_test)))

Shape X_train (150, 318764)
Shape X_test (30, 318764)
Shape y_train (150,)
Shape y_test (30,)


Creamos nuestro perceptron multicapa

In [9]:
def mlp_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=64, kernel_initializer=init, activation='relu'))
    model.add(Dense(2, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Ahora definimos nuestra red neuronal convolucional

In [10]:
def conv_model(optimizer='rmsprop', init='glorot_uniform'):
    # CREO MIS CONVOLUCIONES
    model = Sequential()
    model.add(Convolution1D(16, 5, strides=1, padding='same', kernel_initializer=init, activation='relu', input_shape=(32, 1)))
    model.add(Convolution1D(32, 5, strides=1, padding='same', kernel_initializer=init, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, kernel_initializer=init, activation='relu'))
    model.add(Dense(64, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Ahora el siguiente paso es definir nuestro preprocesador de la entrada y nuestro clasificador. Para esta tarea vamos a utilizar Pipelines que se encargan de enviar mensajes de manera secuencial a los objetos que definamos en el Pipeline. En este caso vamos a pedirle el preproceso al objeto encargado de calcular las componentes principales, las cuales van a alimentar a nuestro clasificador.

In [11]:
class Reshape(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.reshape((150, 32, 1))

In [12]:
#mlp_model = KerasClassifier(build_fn=mlp_model, verbose=0)
conv_model = KerasClassifier(build_fn=conv_model, verbose=0)

estimators = [("reduce_dim", PCA()), ("reshaper", Reshape()), ('clf', conv_model)]
pipeline = Pipeline(estimators)

Vamos a utilizar GridSearch para probar distintas configuraciones de parámetros sobre PCA y la red neuronal.

In [13]:
N_COMPONENTS = [32]
OPTIMIZERS = ['adam']
EPOCHS = [10]
BATCHES = [10]
INIT = ['glorot_uniform']

Creamos nuestro diccionario en el formato que toma GridSearchCV para efectivamente ejecutar las diferentes configuraciones

In [14]:
param_grid = [
    {
        'reduce_dim__n_components': N_COMPONENTS,
        'clf__optimizer': OPTIMIZERS,
        'clf__epochs': EPOCHS,
        'clf__batch_size': BATCHES,
        'clf__init': INIT
    },
]

Para ver que también generaliza nuestro clasificador vamos a utilizar KFold Cross-Validation, en particular con 10 folds.

In [15]:
kfold = KFold(n_splits=5, shuffle=True)

Ahora nos falta fittear nuestro algoritmo y calcular los resultados sobre el conjunto de testeo

In [16]:
grid_search = GridSearchCV(estimator=pipeline, cv=kfold, param_grid=param_grid)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

ValueError: cannot reshape array of size 3840 into shape (150,32,1)

In [ ]:
best_params = grid_search.best_params_
best_n_components = best_params['reduce_dim__n_components']
print("La mejor mejor configuración de parámetros es: \n" + "Cantidad de componentes principales: " + str(best_n_components))

Una vez realizado esto, creamos los gráficos. Para esto creamos dos métodos, uno para la configuración del gráfico (es decir, necesitamos parsear los resultados obtenidos por PCA y KNN de manera que PlotLy los entienda y pueda graficar) y finalmente un método que nos permita graficar usando PlotLy.

In [ ]:
def graph_config(legends, results):
    old_n_neighbors = legends[0]['knn__n_neighbors']
    set_of_lists_with_results = []
    list_with_results = []
    graph_names = []
    for index, legend in enumerate(legends):
        new_n_neighbors = legend['knn__n_neighbors']

        if new_n_neighbors == old_n_neighbors:
            list_with_results.append(results[index])
        else:
            graph_names.append(old_n_neighbors)
            set_of_lists_with_results.append(list_with_results)
            list_with_results = [results[index]]
            old_n_neighbors = new_n_neighbors
    graph_names.append(new_n_neighbors)
    set_of_lists_with_results.append(list_with_results)
    return graph_names, set_of_lists_with_results

def graph(N_COMPONENTS, graph_names, set_of_lists_with_results):
    traces = []
    for index, set in enumerate(set_of_lists_with_results):
        x = N_COMPONENTS
        y = set
        name = "cantidad de vecinos = " + str(graph_names[index])
        traces.append(Scatter(x=x, y=y, name=name))
    layout = Layout(
        xaxis=dict(
            title='Cantidad de componentes principales',
            type='log',
            autorange=True
        ),
        yaxis=dict(
            title='% Accuracy',
            type='log',
            autorange=True
        ),
        title="Medida de performance - Accuracy"
    )
    figure = Figure(data=traces, layout=layout)
    plotly.offline.iplot(figure)

Una vez hecho esto, pasemos a graficar:

In [ ]:
#graph_names, set_of_lists_with_results = graph_config(legends, results)
graph(N_COMPONENTS, ["legend"], [results])